In [ ]:
import torch
import numpy as np
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
print(train.shape)
train.head(4)


(26457, 20)


,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0


In [ ]:
print(train['income_type'].drop_duplicates())
print('-----------------------------------')
print(train['edu_type'].drop_duplicates())
print('-----------------------------------')
print(train['family_type'].drop_duplicates())
print('-----------------------------------')
print(train['house_type'].drop_duplicates())
print('-----------------------------------')
print(train['occyp_type'].drop_duplicates())


0       Commercial associate
2                    Working
4              State servant
14                 Pensioner
2990                 Student
Name: income_type, dtype: object
-----------------------------------
0                   Higher education
1      Secondary / secondary special
51                 Incomplete higher
185                  Lower secondary
676                  Academic degree
Name: edu_type, dtype: object
-----------------------------------
0                  Married
1           Civil marriage
6                Separated
14    Single / not married
46                   Widow
Name: family_type, dtype: object
-----------------------------------
0      Municipal apartment
1        House / apartment
15            With parents
29         Co-op apartment
130       Rented apartment
184       Office apartment
Name: house_type, dtype: object
-----------------------------------
0                        NaN
1                   Laborers
2                   Managers
3             

In [ ]:
# 성별 - 차 - (소득분류) - 부동산 소유 - (생활방식) - (업무용 전화 소유 여부)
# 교육 수준 mapping 


In [ ]:
train['gender_car_reality'] = train['gender'] + '_' + train['car'] + '_' + train['reality']
test['gender_car_reality'] = test['gender'] + '_' + test['car'] + '_' + test['reality']

train = train.drop(['gender', 'car', 'reality'], 1)
test = test.drop(['gender', 'car', 'reality'], 1)

In [ ]:
train['DAYS_BIRTH'] = abs(train['DAYS_BIRTH']) // 365
test['DAYS_BIRTH'] = abs(test['DAYS_BIRTH']) // 365
train['DAYS_BIRTH']

0        38
1        31
2        52
3        41
4        41
         ..
26452    33
26453    41
26454    27
26455    27
26456    53
Name: DAYS_BIRTH, Length: 26457, dtype: int64

In [ ]:
edu = {'Lower secondary' : 0, 'Secondary / secondary special' : 1, 'Incomplete higher' : 2, 'Higher education' : 3, 'Academic degree' : 4}

train['edu_type'] = train['edu_type'].map(edu)
test['edu_type'] = test['edu_type'].map(edu)

train

,index,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,gender_car_reality
0,0,0,202500.0,Commercial associate,3,Married,Municipal apartment,38,-4709,1,0,0,0,NaN,2.0,-6.0,1.0,F_N_N
1,1,1,247500.0,Commercial associate,1,Civil marriage,House / apartment,31,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0,F_N_Y
2,2,0,450000.0,Working,3,Married,House / apartment,52,-4434,1,0,1,0,Managers,2.0,-22.0,2.0,M_Y_Y
3,3,0,202500.0,Commercial associate,1,Married,House / apartment,41,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0,F_N_Y
4,4,0,157500.0,State servant,3,Married,House / apartment,41,-2105,1,0,0,0,Managers,2.0,-26.0,2.0,F_Y_Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,2,225000.0,State servant,1,Married,House / apartment,33,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0,F_N_N
26453,26453,1,180000.0,Working,3,Separated,House / apartment,41,-2475,1,0,0,0,NaN,2.0,-47.0,2.0,F_N_Y
26454,26454,0,292500.0,Working,1,Civil marriage,With parents,27,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0,F_Y_N
26455,26455,0,171000.0,Working,2,Single / not married,House / apartment,27,-107,1,0,0,0,Laborers,1.0,-59.0,2.0,M_N_Y


In [ ]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()

train_ft = pd.get_dummies(train['family_type'])
test_ft = pd.get_dummies(test['family_type'])
train = pd.concat([train, train_ft], axis=1)
test = pd.concat([test, test_ft], axis=1)


train_it = pd.get_dummies(train['income_type'])
test_it = pd.get_dummies(test['income_type'])
train = pd.concat([train, train_it], axis=1)
test = pd.concat([test, test_it], axis=1)


train = train.drop(['income_type', 'house_type', 'family_type'], 1)
test = test.drop(['income_type', 'house_type', 'family_type'] ,1)


train

,index,child_num,income_total,edu_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,gender_car_reality,Civil marriage,Married,Separated,Single / not married,Widow,Commercial associate,Pensioner,State servant,Student,Working
0,0,0,202500.0,3,38,-4709,1,0,0,0,NaN,2.0,-6.0,1.0,F_N_N,0,1,0,0,0,1,0,0,0,0
1,1,1,247500.0,1,31,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0,F_N_Y,1,0,0,0,0,1,0,0,0,0
2,2,0,450000.0,3,52,-4434,1,0,1,0,Managers,2.0,-22.0,2.0,M_Y_Y,0,1,0,0,0,0,0,0,0,1
3,3,0,202500.0,1,41,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0,F_N_Y,0,1,0,0,0,1,0,0,0,0
4,4,0,157500.0,3,41,-2105,1,0,0,0,Managers,2.0,-26.0,2.0,F_Y_Y,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,2,225000.0,1,33,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0,F_N_N,0,1,0,0,0,0,0,1,0,0
26453,26453,1,180000.0,3,41,-2475,1,0,0,0,NaN,2.0,-47.0,2.0,F_N_Y,0,0,1,0,0,0,0,0,0,1
26454,26454,0,292500.0,1,27,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0,F_Y_N,1,0,0,0,0,0,0,0,0,1
26455,26455,0,171000.0,2,27,-107,1,0,0,0,Laborers,1.0,-59.0,2.0,M_N_Y,0,0,0,1,0,0,0,0,0,1


In [ ]:
train= train.dropna()


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df = pd.concat([train['gender_car_reality'], test['gender_car_reality']])
le.fit(df)
train['gender_car_reality'] = le.transform(train['gender_car_reality'])
test['gender_car_reality'] = le.transform(test['gender_car_reality'])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
test = test.fillna('0')

In [ ]:
test

,index,child_num,income_total,edu_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,gender_car_reality,Civil marriage,Married,Separated,Single / not married,Widow,Commercial associate,Pensioner,State servant,Student,Working
0,26457,0,112500.0,1,60,365243,1,0,1,0,0,2.0,-60.0,6,1,0,0,0,0,0,1,0,0,0
1,26458,0,135000.0,3,51,-8671,1,0,1,0,Core staff,2.0,-36.0,1,0,1,0,0,0,0,0,1,0,0
2,26459,0,69372.0,1,43,-217,1,1,1,0,Laborers,2.0,-40.0,1,0,1,0,0,0,0,0,0,0,1
3,26460,0,112500.0,1,52,-2531,1,1,0,0,Drivers,2.0,-41.0,6,0,1,0,0,0,1,0,0,0,0
4,26461,0,225000.0,3,48,-9385,1,1,0,0,Managers,2.0,-8.0,3,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36452,0,202500.0,2,50,-5434,1,1,1,0,Accountants,2.0,-19.0,3,0,1,0,0,0,0,0,0,0,1
9996,36453,0,202500.0,1,29,-1315,1,1,0,0,Laborers,2.0,-34.0,7,1,0,0,0,0,0,0,0,0,1
9997,36454,0,292500.0,1,57,-14018,1,0,0,0,Medicine staff,2.0,-55.0,1,0,1,0,0,0,0,0,0,0,1
9998,36455,0,180000.0,1,45,-1085,1,0,1,0,0,2.0,-33.0,2,0,1,0,0,0,1,0,0,0,0


In [ ]:
le2 = LabelEncoder()
df = pd.concat([train['occyp_type'], test['occyp_type']])
le2.fit(df)
train['occyp_type'] = le2.transform(train['occyp_type'])
test['occyp_type'] = le2.transform(test['occyp_type'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
train['income_total'] = train['income_total'] /100
test['income_total'] = test['income_total'] / 100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:

x_train = train.drop(['credit' , 'index'], 1)
y_train = train['credit']

x_train.shape

(18286, 23)

In [ ]:

x_test = test.iloc[:, 1:] 
x_test


,child_num,income_total,edu_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,gender_car_reality,Civil marriage,Married,Separated,Single / not married,Widow,Commercial associate,Pensioner,State servant,Student,Working
0,0,1125.00,1,60,365243,1,0,1,0,0,2.0,-60.0,6,1,0,0,0,0,0,1,0,0,0
1,0,1350.00,3,51,-8671,1,0,1,0,4,2.0,-36.0,1,0,1,0,0,0,0,0,1,0,0
2,0,693.72,1,43,-217,1,1,1,0,9,2.0,-40.0,1,0,1,0,0,0,0,0,0,0,1
3,0,1125.00,1,52,-2531,1,1,0,0,5,2.0,-41.0,6,0,1,0,0,0,1,0,0,0,0
4,0,2250.00,3,48,-9385,1,1,0,0,11,2.0,-8.0,3,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,2025.00,2,50,-5434,1,1,1,0,1,2.0,-19.0,3,0,1,0,0,0,0,0,0,0,1
9996,0,2025.00,1,29,-1315,1,1,0,0,9,2.0,-34.0,7,1,0,0,0,0,0,0,0,0,1
9997,0,2925.00,1,57,-14018,1,0,0,0,12,2.0,-55.0,1,0,1,0,0,0,0,0,0,0,1
9998,0,1800.00,1,45,-1085,1,0,1,0,0,2.0,-33.0,2,0,1,0,0,0,1,0,0,0,0


In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, shuffle=True, stratify=y_train)


In [ ]:
from lightgbm import LGBMClassifier
import random
random.seed(42)
lgb_models={}

for fold in range(6):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(x_train, y_train, 
            eval_set=[(x_train, y_train), (x_val, y_val)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')


====================================1============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.634201	valid_1's multi_logloss: 0.763563
[200]	training's multi_logloss: 0.538333	valid_1's multi_logloss: 0.753743
Early stopping, best iteration is:
[256]	training's multi_logloss: 0.497788	valid_1's multi_logloss: 0.752258


====================================2============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.634201	valid_1's multi_logloss: 0.763563
[200]	training's multi_logloss: 0.538333	valid_1's multi_logloss: 0.753743
Early stopping, best iteration is:
[256]	training's multi_logloss: 0.497788	valid_1's multi_logloss: 0.752258


====================================3============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.634201	val

In [ ]:
print(x_train.shape)
print(x_test.shape)

(13714, 23)
(10000, 23)


In [ ]:
submission.iloc[:,1:]=0
for fold in range(5):
    submission.iloc[:,1:] += lgb_models[fold].predict_proba(x_test)/5

In [ ]:
submission.to_csv('submission.csv', index = False)